In [ ]:
import numpy as np
import pickle

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/lane-detection/full_CNN_train.p
/kaggle/input/lane-detection/full_CNN_labels.p


In [ ]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

In [ ]:
pip install tensorflow-gpu

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import tensorflow as tf

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.config.set_soft_device_placement(True)


Num GPUs Available:  1


In [ ]:
train_images = pickle.load(open("/kaggle/input/lane-detection/full_CNN_train.p", "rb" ))

labels = pickle.load(open("/kaggle/input/lane-detection/full_CNN_labels.p", "rb" ))

# Make into arrays as the neural network wants these
train_images = np.array(train_images)
labels = np.array(labels)


In [ ]:
# Normalize labels 
labels= labels/255

train_images, labels = shuffle(train_images, labels)

X_train, X_val, y_train, y_val = train_test_split(train_images, labels, test_size=0.128)

# Batch size, epochs and pool size below are all paramaters to fiddle with for optimization
batch_size = 128
epochs = 25
pool_size = (2, 2)
input_shape = X_train.shape[1:]

In [ ]:
train_datagen = ImageDataGenerator(channel_shift_range=0.2)
train_datagen.fit(X_train)

In [ ]:
model=tf.keras.models.Sequential([
# Normalizes incoming inputs. First layer needs the input shape to work
tf.keras.layers.BatchNormalization(input_shape=input_shape),
tf.keras.layers.Conv2D(8, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv1'),
tf.keras.layers.Conv2D(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv2'),
# Pooling 1
tf.keras.layers.MaxPooling2D(pool_size=pool_size),

tf.keras.layers.Conv2D(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv3'),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Conv2D(32, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv4'),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Conv2D(32, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv5'),
tf.keras.layers.Dropout(0.2),
 # Pooling 2
tf.keras.layers.MaxPooling2D(pool_size=pool_size),

tf.keras.layers.Conv2D(64, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv6'),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Conv2D(64, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv7'),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Conv2D(64, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv8'),
# Pooling 3
tf.keras.layers.MaxPooling2D(pool_size=pool_size),
# Upsample 1
tf.keras.layers.UpSampling2D(size=pool_size),
#Deconv
tf.keras.layers.Conv2DTranspose(128, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv1'),
tf.keras.layers.Conv2DTranspose(64, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv2'),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Conv2DTranspose(64, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv3'),
tf.keras.layers.Dropout(0.25),
# Upsample 2
tf.keras.layers.UpSampling2D(size=pool_size),

tf.keras.layers.Conv2DTranspose(32, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv4'),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Conv2DTranspose(32, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv5'),
# Upsample 3
tf.keras.layers.UpSampling2D(size=pool_size),

tf.keras.layers.Conv2DTranspose(32, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv6'),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Conv2DTranspose(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv7'),
tf.keras.layers.Dropout(0.25),
tf.keras.layers.Conv2DTranspose(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv8'),
tf.keras.layers.Conv2DTranspose(1, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv9Finalone')
])

In [ ]:
# Compiling and training the model
model.compile(optimizer='Adam', loss='mean_squared_error',metrics=['accuracy'])

model.fit(train_datagen.flow(X_train, y_train, batch_size=batch_size), 
                steps_per_epoch=len(X_train)/batch_size,
                epochs=epochs, verbose=1, validation_data=(X_val, y_val))

Epoch 1/25
86/86 [==============================] - 25s 168ms/step - loss: 0.1117 - accuracy: 0.8485 - val_loss: 0.0292 - val_accuracy: 0.9376
Epoch 2/25
86/86 [==============================] - 12s 135ms/step - loss: 0.0233 - accuracy: 0.9441 - val_loss: 0.0152 - val_accuracy: 0.9523
Epoch 3/25
86/86 [==============================] - 12s 140ms/step - loss: 0.0134 - accuracy: 0.9537 - val_loss: 0.0101 - val_accuracy: 0.9559
Epoch 4/25
86/86 [==============================] - 12s 137ms/step - loss: 0.0107 - accuracy: 0.9556 - val_loss: 0.0086 - val_accuracy: 0.9571
Epoch 5/25
86/86 [==============================] - 12s 138ms/step - loss: 0.0094 - accuracy: 0.9565 - val_loss: 0.0088 - val_accuracy: 0.9569
Epoch 6/25
86/86 [==============================] - 13s 144ms/step - loss: 0.0088 - accuracy: 0.9568 - val_loss: 0.0078 - val_accuracy: 0.9575
Epoch 7/25
86/86 [==============================] - 12s 135ms/step - loss: 0.0084 - accuracy: 0.9571 - val_loss: 0.0075 - val_accuracy: 0.9577

In [ ]:
model.trainable = False
model.save('./lanemodel.h5')

In [ ]:
model.save('./lane_modelpth.pth')

In [ ]:
model.save('./lane_modeltar.tar')

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 80, 160, 3)        12        
_________________________________________________________________
Conv1 (Conv2D)               (None, 78, 158, 8)        224       
_________________________________________________________________
Conv2 (Conv2D)               (None, 76, 156, 16)       1168      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 38, 78, 16)        0         
_________________________________________________________________
Conv3 (Conv2D)               (None, 36, 76, 16)        2320      
_________________________________________________________________
dropout (Dropout)            (None, 36, 76, 16)        0         
_________________________________________________________________
Conv4 (Conv2D)               (None, 34, 74, 32)        4